<h1 style="font-size:200%; font-family:cursive; color:white;">1. Import Required Libraries & Dataset</h1>

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
base_path = Path(os.getcwd()).parent / 'Modules'
sys.path.append(str(base_path))
from KeyFrameExtractorClass import KeyFrameExtractor

In [2]:
df_videos = pd.DataFrame(columns=['video_name', 'video_path', 'translation'])
videos_path = Path(os.getcwd()).parent / 'Resources' / 'Datasets' / 'Videos'
df_translation = pd.read_csv(str(videos_path.parent.parent / 'Translations' / 'how2sign_train.csv'), sep='\t', usecols=['VIDEO_NAME', 'SENTENCE']).groupby('VIDEO_NAME')

In [3]:
for video in os.listdir(str(videos_path)):
    video_path = videos_path / video
    video_name = video.removeprefix('filtered_').removesuffix('.mp4')
    sentence = df_translation.get_group(video_name)['SENTENCE'].str.cat(sep=' ')
    new_row = pd.DataFrame({'video_name': [video_name], 'video_path': [video_path], 'translation': [sentence]})
    df_videos = pd.concat([df_videos, new_row])
df_videos.head()

,video_name,video_path,translation
0,-00cp1iGiDw-5-rgb_front,c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...,If you like to find more about my services you...
0,-08ZGCviCm4-5-rgb_front,c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...,Hello welcome my name is Julio Nutt and I am a...
0,-1KpdDGPCq4-5-rgb_front,c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...,"All right, the drink we're about to make is ca..."
0,-1XUnputrgk-5-rgb_front,c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...,"Okay, another thing that for me is important. ..."
0,-3sFcDBFxc8-5-rgb_front,c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...,"All right, we're ready to start our fire. What..."


In [4]:
import torch
import torch.nn as nn
from torchvision import transforms
from transformers import ViTModel, GPT2LMHeadModel, GPT2Tokenizer


#specify GPU
device = torch.device("cpu")

c:\Users\matia\OneDrive\Escritorio\SignAI-ML\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Check if Pytorch is using GPU

In [5]:
torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.current_device(), torch.cuda.device(0), torch.cuda.get_device_name(0), device

AssertionError: Torch not compiled with CUDA enabled

In [6]:
from sklearn.model_selection import train_test_split
X_df = df_videos['video_path'].to_frame()
y_df = df_videos['translation'].to_frame()
seed = 31991
batch_size = 32
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=(100/500), random_state=seed)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=(100/400), random_state=seed)

In [7]:
X_train.head(), y_train.head(), X_val.head(), y_val.head(), X_test.head(), y_test.head()

(                                          video_path
 0  c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...
 0  c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...
 0  c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...
 0  c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...
 0  c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...,
                                          translation
 0  Hi! Let it drip off a little bit, put it on a ...
 0  I'm Ivan Madar. Skiing is done from your waste...
 0  Hi, this Sean Hobson, and right now we're work...
 0  Developing reading skills isn't it important t...
 0  We have 2 different receipts to show you a pea...,
                                           video_path
 0  c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...
 0  c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...
 0  c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...
 0  c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...
 0  c:\Users\matia\OneDrive\Escritorio\SignAI-ML\A...,
                         

In [8]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((300, 1), (300, 1), (100, 1), (100, 1), (100, 1), (100, 1))

<h1 style="font-size:200%; font-family:cursive; color:white;">3. Import ViT</h1>

In [9]:
# import BERT-base pretrained model
model = ViTModel.from_pretrained('google/vit-base-patch16-224')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
tokens_label_train = tokenizer(
    y_train['translation'].tolist(),
    padding=True,
    return_tensors="pt"
)

tokens_label_val = tokenizer(
    y_val['translation'].tolist(),
    padding=True,
    return_tensors="pt"
)

tokens_label_test = tokenizer(
    y_test['translation'].tolist(),
    padding=True,
    return_tensors="pt"
)

<h1 style="font-size:200%; font-family:cursive; color:white;">5. List to Tensors</h1>

In [11]:
y_train['token_ids'] = tokens_label_train['input_ids'].tolist()
y_val['token_ids'] = tokens_label_val['input_ids'].tolist()
y_test['token_ids'] = tokens_label_test['input_ids'].tolist()

<h1 style="font-size:200%; font-family:cursive; color:white;">7. Model Architecture</h1>

In [ ]:
vocab_size = tokenizer.vocab_size

class BERT_Arch(nn.Module):
    def __init__(self, bert, vocab_size, max_len=512):
        super(BERT_Arch, self).__init__()
        
        self.bert = bert 
        self.reduce = reduce
        self.vocab_size = vocab_size
        self.max_len = max_len
        
        # Dropout layer
        self.dropout = nn.Dropout(0.1)
        
        # Dense layer (Output layer)
        self.fc = nn.Linear(bert.config.hidden_size, vocab_size)

    def forward(self, seq_input: torch.Tensor):
        # # Aplicar reducción de dimensionalidad
        # seq_input = seq_input.unsqueeze(1)
        # print(seq_input.shape)
        # seq_input = self.reduce(seq_input)  # (batch_size, 512)
        # print(seq_input.shape)
        # # Crear máscara de atención
        # attention_mask = torch.ones(seq_input.shape[0], seq_input.shape[1]).to(seq_input.device)
        # print(seq_input.shape == attention_mask.shape)
        seq_input = torch.ones(32,512,512,512).half().to(device)
        attention_mask = torch.ones(32,512,512,512).half().to(device)
        print(seq_input.shape, attention_mask.shape)
        outputs = self.bert(inputs_embeds=seq_input, attention_mask=attention_mask, return_dict=True)
        print("salio")
        x = outputs.last_hidden_state  # Usar el último estado oculto

        # Pasar por la capa densa
        logits = self.fc(x)  # logits tiene forma (batch_size, 512, vocab_size)

        return logits
    
    def generate(self, input_ids, max_length):
        self.eval()
        generated_tokens = []

        # Inicializar el estado de entrada para generación
        for _ in range(max_length):
            outputs = self.forward(input_ids)
            next_token = torch.argmax(outputs[:, -1, :], dim=-1)  # Obtener el token más probable
            generated_tokens.append(next_token.item())
            
            # Actualizar input_ids para el siguiente token
            input_ids = torch.cat([input_ids, next_token.unsqueeze(1)], dim=1)

            # Salir si se alcanza el token de fin de secuencia
            if next_token.item() == tokenizer.eos_token_id:
                break
        return generated_tokens

# Inicializar el modelo con BERT y vocab_size
model = BERT_Arch(bert, vocab_size)

# Mover el modelo a la GPU
model = model.to(device)

# Definir el optimizador y la función de pérdida
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)
cross_entropy = nn.CrossEntropyLoss()

# Número de épocas de entrenamiento
epochs = 10

<h1 style="font-size:200%; font-family:cursive; color:white;">8. Fine - Tune</h1>

In [ ]:
import wandb
wandb.login()

In [24]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(preds, references):
    smoothie = SmoothingFunction().method4  # Smoothing method to avoid zero scores for short sentences
    # Calcula el BLEU score para cada predicción en el batch
    scores = []
    for pred, ref in zip(preds, references):
        score = sentence_bleu([ref], pred, smoothing_function=smoothie)
        scores.append(score)
    return scores

# Function to convert token ids back to words (predictions and labels)
def decode_predictions(predictions, tokenizer):
    decoded_preds = []
    for pred in predictions:
        # Convert token IDs to tokens (words) using the tokenizer's decode method
        decoded = tokenizer.decode(pred, skip_special_tokens=True)
        decoded_preds.append(decoded)
    return decoded_preds

In [25]:
def train(
        model: torch.nn.Module, 
        train_loader: DataLoader, 
        optimizer: torch.optim.Optimizer, 
        criterion: torch.nn.Module, 
        device: torch.device, 
        epoch: int):
    
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0
    
    all_preds = []
    all_refs = []

    for batch_idx, (inputs, mask, labels) in enumerate(train_loader):
        inputs, mask, labels = inputs.to(device), mask.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)  # (batch_size, seq_length, vocab_size)
        loss = criterion(outputs.view(-1, outputs.size(-1)), labels.view(-1))
        
        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Cálculo de métricas
        running_loss += loss.item()
        total += labels.numel()
        _, predicted = torch.max(outputs.view(-1, outputs.size(-1)), 1)
        correct += (predicted == labels.view(-1)).sum().item()

        # Decodificación para BLEU
        all_preds.extend(predicted.tolist())
        all_refs.extend(labels.view(-1).tolist())
        
        # Logs en consola
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')
        
        # Logs en W&B
        wandb.log({
            "epoch": epoch,
            "batch_idx": batch_idx,
            "loss": loss.item(),
            "accuracy_batch": 100 * correct / total
        })
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    # Calcular BLEU
    train_bleu = calculate_bleu(all_preds, all_refs)

    print(f"Epoch [{epoch}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
    
    return epoch_loss, train_bleu

def evaluate(
        model: torch.nn.Module, 
        val_loader: DataLoader, 
        criterion: torch.nn.Module, 
        device: torch.device, 
        epoch: int):
    
    model.eval()
    running_loss = 0.0
    total = 0
    correct = 0
    
    all_preds = []
    all_refs = []

    with torch.no_grad():
        for batch_idx, (inputs, mask, labels) in enumerate(val_loader):
            inputs, mask, labels = inputs.to(device), mask.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs.view(-1, outputs.size(-1)), labels.view(-1))
            
            # Cálculo de métricas
            running_loss += loss.item()
            total += labels.numel()
            _, predicted = torch.max(outputs.view(-1, outputs.size(-1)), 1)
            correct += (predicted == labels.view(-1)).sum().item()

            # Decodificación para BLEU
            all_preds.extend(predicted.tolist())
            all_refs.extend(labels.view(-1).tolist())
            
            # Logs en consola
            if batch_idx % 100 == 0:
                print(f'Eval Step [{batch_idx}/{len(val_loader)}], Loss: {loss.item():.4f}')
            
            # Logs en W&B
            wandb.log({
                "eval_batch_idx": batch_idx,
                "eval_loss": loss.item(),
                "eval_accuracy_batch": 100 * correct / total
            })
    
    epoch_loss = running_loss / len(val_loader)
    epoch_accuracy = 100 * correct / total

    # Calcular BLEU
    valid_bleu = calculate_bleu(all_preds, all_refs)

    print(f"Validation Epoch [{epoch}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
    
    return epoch_loss, valid_bleu


In [ ]:
# Inicialización de variables para seguimiento del mejor modelo
best_valid_loss = float('inf')
best_bleu_score = 0

# Listas para almacenar loss y BLEU de cada epoch
train_losses = []
valid_losses = []
train_bleu_scores = []
valid_bleu_scores = []

# Configuración de hiperparámetros
wandb.config = {
    "epochs": epochs,
    "batch_size": batch_size,
    "learning_rate": optimizer.param_groups[0]['lr']
}

# Ciclo de entrenamiento y evaluación por cada epoch
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")

    # Entrenar el modelo
    train_loss, train_bleu = train(model, train_dataloader, optimizer, cross_entropy, device, epoch)
    
    # Evaluar el modelo
    valid_loss, valid_bleu = evaluate(model, val_dataloader, cross_entropy, device, epoch)

    # Guardar el mejor modelo basado en la validación de loss
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best_model.pt')
        print(f"Modelo guardado en epoch {epoch + 1} con loss de validación {valid_loss:.4f}")

    # Guardar loss y BLEU para entrenamiento y validación
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    train_bleu_scores.append(train_bleu)
    valid_bleu_scores.append(valid_bleu)

    # Mostrar estadísticas de entrenamiento y validación
    print(f"Training Loss: {train_loss:.4f} | Training BLEU: {train_bleu:.4f}")
    print(f"Validation Loss: {valid_loss:.4f} | Validation BLEU: {valid_bleu:.4f}")
    
    # Registros de la época en W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_bleu": train_bleu,
        "valid_loss": valid_loss,
        "valid_bleu": valid_bleu
    })

# Finaliza el seguimiento de W&B
wandb.finish()

In [ ]:
#load weights of best model
path = 'best_model.pt'
model.load_state_dict(torch.load(path))

<h1 style="font-size:200%; font-family:cursive; color:white;">9. Make Predictions</h1>

In [ ]:
# get predictions for test data
with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()

In [ ]:
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))